In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import datetime 

import smtplib

url="https://merolagani.com/StockQuote.aspx"
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36" 
 }
page = requests.get(url)
soup1=BeautifulSoup(page.text,'lxml')
#soup2=BeautifulSoup(soup1.prettify(),"html.parser")

#lists=soup1.find_all('td',class_="text-right").text

#lists= soup1.find_all('td', attrs = {'class':"text-right"}).get_text

#print(lists)
lists2=soup1.find('table',class_='table table-bordered table-striped table-hover sortable')
#print(lists2)
headers=[]
for i in lists2.find_all('th'): 
    
        title=i.text
        headers.append(title)

import re
mydata = pd.DataFrame(columns=['Num','Symbol','LTP','Change','High','Low','Open','Qty','Turnover'])

for j in lists2.find_all('tr')[1:]:
    
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(mydata)
        mydata.loc[length] = row

mydata.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
# Drop “#” column
#mydata.drop('\n#', inplace=True, axis=1)
# Export to csv
mydata.to_csv('dataset_stock.csv', index=False)
# Try to read csv
mydata2 = pd.read_csv('dataset_stock.csv')            
print(mydata2)      

    Num  Symbol     LTP  Change    High     Low    Open     Qty       Turnover
0     1  ACLBSL  720.00    1.27  722.00  700.10  711.00     247     176,791.50
1     2    ADBL  311.70   -1.05  315.40  311.10  312.00  16,382   5,113,726.40
2     3    AHPC  272.80    0.29  277.00  270.00  275.00  23,219   6,302,353.60
3     4     AIL  409.00   -1.45  412.00  406.70  406.70   5,684   2,322,068.00
4     5   AKJCL  171.10   -0.23  173.00  171.00  171.60  14,581   2,506,644.50
..  ...     ...     ...     ...     ...     ...     ...     ...            ...
95   96     MBL  221.00   -0.45  222.50  218.10  218.10  11,315   2,503,283.90
96   97     MDB  386.00   -0.52  393.00  382.10  392.90  10,280   3,966,913.20
97   98    MEGA  214.40    0.14  216.00  213.60  214.00  95,761  20,521,846.40
98   99     MEN  741.70    0.83  741.70  721.20  721.20     430     315,721.00
99  100    MERO  777.90    0.76  780.00  760.20  772.00   5,176   3,981,528.40

[100 rows x 9 columns]


In [7]:
import pyodbc
import pandas as pd

# insert data from csv file into dataframe.
# working directory for csv file: type "pwd" in Azure Data Studio or Linux
# working directory in Windows c:\users\username
df = pd.read_csv("C:\\Users\\editd\\Desktop\\course_dataprogramming\\Final_project\\dataset_stock.csv")
# Some other example server values are

#cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-G6J8J7M;'
                      'Database=BDAT_1004;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

#cursor.execute('CREATE TABLE NepalStock (Num,Symbol,LTP,Change,High_,Low_,Open_,Qty,Turnover)')
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():
     cursor.execute("INSERT INTO NepalStock (Num,Symbol,LTP,Change,High_,Low_,Open_,Qty,Turnover) values(?,?,?,?,?,?,?,?,?)",row.Num,row.Symbol,row.LTP,row.Change,row.High,row.Low,row.Open,row.Qty,row.Turnover)
conn.commit()
cursor.close()

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Error converting data type nvarchar to float. (8114) (SQLExecDirectW)')

In [5]:
import sqlite3
import sqlalchemy
con=sqlite3.connect('model.db')
engine=sqlalchemy.create_engine('sqlite:///model.db')
cur=con.cursor()

cur.execute('CREATE TABLE NepalStock (Num,Symbol,LTP,Change,High,Low,Open,Qty,Turnover)')

mydata2.to_sql('NepalStock',engine,if_exists='append',index=False)

cur.execute("select * from NepalStock")

for record in cur:
    print(record)
#con.commit()
#cursor.close()
#con.close()    



(1, 'ACLBSL', '695.00', -1.42, '725.00', '695.00', '695.00', '625', '436,311.30')
(2, 'ADBL', '319.00', 0.63, '319.90', '315.00', '317.00', '6,331', '2,008,843.30')
(3, 'AHPC', '275.00', 1.1, '276.90', '267.00', '276.90', '34,101', '9,265,760.30')
(4, 'AKJCL', '176.50', 0.91, '178.20', '173.60', '177.00', '17,780', '3,119,016.80')
(5, 'AKPL', '352.00', -0.28, '354.90', '347.20', '350.00', '20,132', '7,072,207.40')
(6, 'ALBSL', '900.00', -1.1, '915.00', '891.80', '910.00', '711', '636,927.00')
(7, 'ALICL', '482.00', -0.41, '486.80', '480.00', '480.00', '5,907', '2,860,826.50')
(8, 'API', '217.00', 0.93, '218.00', '213.00', '214.60', '68,691', '14,857,573.70')
(9, 'AVYAN', '714.00', -0.7, '715.00', '708.30', '708.30', '2,013', '1,435,405.30')
(10, 'BARUN', '213.90', 0.33, '216.00', '213.00', '216.00', '970', '208,049.50')
(11, 'BBC', '3,330.00', 1.31, '3,330.00', '3,330.00', '3,330.00', '10', '33,300.00')
(12, 'BFC', '319.00', 0.0, '319.20', '313.00', '313.10', '5,424', '1,719,216.00')
(